# More information


1. This notebook is my original work on the brain tumor classification task. It has undergone many changes based on my teacher's recommendations. The first version suffered from overfitting, which I addressed using various methods in the nb2_optimization_model notebook. You can check it for more details.

2. You can test my model using the nb0_predict_new_img notebook to test with new images in the pred folder. Detailed instructions are provided in that notebook.

# Import library

In [1]:
import cv2
import os
import random
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from tensorflow import keras
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
from keras.utils import normalize
from PIL import Image
from sklearn.model_selection import train_test_split

### Avoid difference result after rerun notebook

In [2]:
seed = 99
tf.random.set_seed(seed)
np.random.seed(seed)
random.seed(seed)

# Reading dataset

In [3]:
no_dir = os.listdir('./data_no/data_no/NO/')
yes_dir = os.listdir('./data_yes/data_yes/YES/')

In [4]:
len(no_dir)

535

In [5]:
len(yes_dir)

557

In [6]:
data_set,label = [],[]
for i,cur_img_dir in enumerate(no_dir):
    #check type of image
    if cur_img_dir.split('.')[1]=='jpg':
        img = cv2.imread('./data_no/data_no/NO/'+cur_img_dir)
        img = Image.fromarray(img,'RGB')
        img = img.resize((64,64))
        data_set.append(np.array(img))
        label.append(0)

In [7]:
for i,cur_img_dir in enumerate(yes_dir):
    #check type of image
    if cur_img_dir.split('.')[1]=='jpg':
        img = cv2.imread('./data_yes/data_yes/YES/'+cur_img_dir)
        img = Image.fromarray(img,'RGB')
        img = img.resize((64,64))
        data_set.append(np.array(img))
        label.append(1)

In [8]:
data_set = np.array(data_set)
label = np.array(label)
data_set.shape

(1092, 64, 64, 3)

In [9]:
label.shape

(1092,)

In [10]:
print(f'yes observe:{sum(label)}, no observe:{len(label)-sum(label)}')

yes observe:557, no observe:535


# Split and normalize data


## 1. Split


In [11]:
x_train,x_test,y_train,y_test = train_test_split(
    data_set,label,
    test_size=0.2,
    random_state=9
    )
x_train,x_val,y_train,y_val = train_test_split(
        x_train,y_train,
    test_size=0.25,
    random_state=9
)


In [12]:
print(f'X train shape: {x_train.shape}\nY train shape: {y_train.shape}\nX test shape: {x_test.shape}\nY test shape: {y_test.shape}\nX validation shape: {x_val.shape}\nY validation shape: {x_val.shape}')

X train shape: (654, 64, 64, 3)
Y train shape: (654,)
X test shape: (219, 64, 64, 3)
Y test shape: (219,)
X validation shape: (219, 64, 64, 3)
Y validation shape: (219, 64, 64, 3)


## 2. Normalize

In [13]:
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import joblib 
# using scaler of nb4
scaler = joblib.load('scaler.pkl') 
# Reshape data to fit with MinMaxScaler
x_train_reshaped = x_train.reshape(-1, x_train.shape[-1])
x_test_reshaped = x_test.reshape(-1, x_test.shape[-1])
x_val_reshaped = x_val.reshape(-1, x_val.shape[-1])

x_train_reshaped = scaler.fit_transform(x_train_reshaped)
x_test_reshaped = scaler.transform(x_test_reshaped)
x_val_reshaped = scaler.transform(x_val_reshaped)

# Reshape to original shape
x_train = x_train_reshaped.reshape(x_train.shape)
x_test = x_test_reshaped.reshape(x_test.shape)
x_val = x_val_reshaped.reshape(x_val.shape)

# Building Model

In [14]:
from keras.models import Sequential 
from tensorflow.keras.optimizers import Adam
from keras.layers import (
    Conv2D,
    MaxPooling2D,
    Activation,
    Dropout,
    Flatten,
    Dense
)

In [15]:
model=Sequential()
INPUT_SIZE = 64

model.add(Conv2D(32,(3,3), input_shape=(INPUT_SIZE,INPUT_SIZE,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(32, (3,3), kernel_initializer='he_uniform'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))


model.add(Conv2D(64,(3,3), kernel_initializer='he_uniform'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(1))
model.add(Activation('sigmoid'))

c:\Users\Admin\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


# Training model

*update 25/08 - adding early stopping method* 

In [16]:
import tensorflow as tf

# Compile the model with accuracy metric
model.compile(
    loss='binary_crossentropy',
    optimizer=tf.keras.optimizers.Adam(amsgrad=True),
    metrics=['accuracy']
)

# Define EarlyStopping callback
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',  # Monitor validation loss
    patience=3,          # Number of epochs with no improvement after which training will be stopped
    restore_best_weights=True  # Restore model weights from the epoch with the best value of the monitored quantity
)

# Fit model
history = model.fit(
    x_train,
    y_train,
    batch_size=16,
    epochs=50,
    validation_data=(x_val, y_val),
    shuffle=False,  # Determine whether to shuffle the training data before each epoch
    callbacks=[early_stopping]  # Add EarlyStopping callback
)

Epoch 1/50
41/41 ━━━━━━━━━━━━━━━━━━━━ 8s 51ms/step - accuracy: 0.6539 - loss: 0.6269 - val_accuracy: 0.7626 - val_loss: 0.4704
Epoch 2/50
41/41 ━━━━━━━━━━━━━━━━━━━━ 2s 35ms/step - accuracy: 0.8628 - loss: 0.3625 - val_accuracy: 0.8402 - val_loss: 0.3854
Epoch 3/50
41/41 ━━━━━━━━━━━━━━━━━━━━ 2s 37ms/step - accuracy: 0.9235 - loss: 0.2331 - val_accuracy: 0.8584 - val_loss: 0.3119
Epoch 4/50
41/41 ━━━━━━━━━━━━━━━━━━━━ 2s 46ms/step - accuracy: 0.9497 - loss: 0.1672 - val_accuracy: 0.8767 - val_loss: 0.2972
Epoch 5/50
41/41 ━━━━━━━━━━━━━━━━━━━━ 2s 38ms/step - accuracy: 0.9528 - loss: 0.1531 - val_accuracy: 0.8447 - val_loss: 0.3211
Epoch 6/50
41/41 ━━━━━━━━━━━━━━━━━━━━ 2s 38ms/step - accuracy: 0.9605 - loss: 0.1417 - val_accuracy: 0.8539 - val_loss: 0.3394
Epoch 7/50
41/41 ━━━━━━━━━━━━━━━━━━━━ 2s 40ms/step - accuracy: 0.9574 - loss: 0.1183 - val_accuracy: 0.8995 - val_loss: 0.2369
Epoch 8/50
41/41 ━━━━━━━━━━━━━━━━━━━━ 2s 39ms/step - accuracy: 0.9712 - loss: 0.0868 - val_accuracy: 0.8858 - v

# Test with new data

In [17]:
y_test_pred = model.predict(x_test)
print(y_test_pred.shape)

7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 64ms/step
(219, 1)


In [18]:
y_pred = (y_test_pred >0.5).astype(int)
y_pred.shape

(219, 1)

In [19]:
y_test_reshape = y_test.reshape(-1,1)
y_test_reshape.shape

(219, 1)

In [20]:
print("Accuracy in test set:", accuracy_score(y_test_reshape, y_pred))
print('Accuracy in validation set:',history.history['val_accuracy'][-1])

Accuracy in test set: 0.958904109589041
Accuracy in validation set: 0.9452054500579834


# Save original model 

**Infor**<br>
Accuracy in test set: 0.958904109589041<br>
Accuracy in validation set: 0.9452054500579834<br>
Name: BrainTurmor_v2<br>

Status: Good

# Save original model 

In [21]:
model.save('BrainTurmor_v1.keras')